In [14]:
import pandas as pd
from statistics import mean, median
import sys
import pprint
import pickle

In [15]:
pp = pprint.PrettyPrinter(indent=4)

In [16]:
batch_sizes = [2, 4, 8, 16]
batching_alg = ['BATCH4', 'BATCHBISECT', 'BATCHSTOP4']
projects = []
num = 1

In [17]:
def get_results(results):
    l = []
    
    for r in results:
        if r == 'passed':
            l.append(1)
        else:
            l.append(0)
            
    return l

In [18]:
def str_to_list(s):
    if s == '[]':
        return [0]
    l = s[1:-1].split(', ')
    l = [int(x) for x in l]
    return l

In [19]:
def get_empty_library():
    global projects
    aici_delay_lib = {}
    
    for p in projects:
        aici_delay_lib[p] = {'BATCHSTOP4':{4:{}, 8:{}, 16:{}}, 'BATCH4':{4:{}}, 'BATCHBISECT':{1:{}, 2:{}, 4:{}, 8:{}, 16:{}}}
    
    return aici_delay_lib
        

In [20]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [21]:
def get_first_failures(df):
    
    results = df['tr_status'].tolist()
    length = len(results)
    verdict = ['keep']
    prev = results[0]
    
    for i in range(1, length):
        if results[i] == 0:
            if prev == 0:
                verdict.append('discard')
                #print(i+1)
            else:
                verdict.append('keep')
        else:
            verdict.append('keep')
        prev = results[i]
    
    df['verdict'] = verdict
    df = df[ df['verdict'] == 'keep' ]
    df.drop('verdict', inplace=True, axis=1)
    return df

In [22]:
def get_delay_from_ci_old(ci, y_test, batch_size):
    
    i = 0        
    delay_list = []
    missed = []
    batch_list = []
    sbs_list = []
    
    b = batch_size-1

    while i < len(ci):

        if ci[i] == 0:

            while len(missed) > 0:
                ind = missed.pop()
                sbs_list.append(i - ind)

            batch_list.append(b)
            b -= 1
            if b == -1:
                b = batch_size - 1
            
            #print(y_test[i], ci[i], sbs_delay, batch_delay, sbs_delay + batch_delay)

        if ci[i] == 1:
            if y_test[i] == 0:
                missed.append(i)

        i += 1

    while len(missed) > 0:
            sbs_list.append(i - missed.pop())
    
    delay_list.extend(sbs_list)
    delay_list.extend(batch_list)
    
    return delay_list

In [ ]:
def get_delay_from_ci(ci, y_test, batch_size):
    
    i = 0        
    delay_list = []
    missed = []
    batch_list = []
    sbs_list = []
    delay_list = []
    
    b = batch_size-1

    while i < len(ci):

        if ci[i] == 0:

            while len(missed) > 0:
                ind = missed.pop()
                sbs_list.append(i - ind)

            batch_list.append(b)
            b -= 1
            if b == -1:
                b = batch_size - 1
            
            #print(y_test[i], ci[i], sbs_delay, batch_delay, sbs_delay + batch_delay)

        if ci[i] == 1:
            if y_test[i] == 0:
                missed.append(i)

        i += 1

    while len(missed) > 0:
            sbs_list.append(i - missed.pop())
    
    delay_list.extend(sbs_list)
    delay_list.extend(batch_list)
    
    return delay_list

In [23]:
def get_delay_lib(aici_results):
    
    aici_delay_lib = get_empty_library()
    
    for x in range(len(aici_results)):
        row = aici_results.iloc[x]
        ci = str_to_list(row['ci'])
        
        batch_size = row['batch_size']
        project = row['project']

        test_file = pd.read_csv('../../data/exact_data/' + project ) 
        test_file['tr_status'] = output_values(test_file['tr_status'])

        y_test = test_file['tr_status'].tolist()
        
        delay_list = get_delay_from_ci(ci, y_test, batch_size)
        #print(delay_list)
        print(median(delay_list), max(delay_list))
        aici_delay_lib[row['project']][row['algorithm']][row['batch_size']]['delays'] = delay_list
    
    return aici_delay_lib
        

In [24]:
def get_lines(aici_results, aici_delay_lib):
    lines = []
    for x in range(len(aici_results)):
        row = aici_results.iloc[x]
        d = aici_delay_lib[row['project']][row['algorithm']][row['batch_size']]['delays']
        m = median(d)
        lines.append([row['project'], row['algorithm'], row['batch_size'], row['builds_reqd'], row['testall_size'], d, m])
    return lines


In [25]:
def start_result_collection(aici_result_file):
    
    global projects
    global num
    
    aici_results = pd.read_csv(aici_result_file)    
    projects = set(aici_results['project'].tolist())
    
    aici_delay_lib = get_delay_lib(aici_results)
    lines = get_lines(aici_results, aici_delay_lib)
    
    num += 1
    
    df = pd.DataFrame(lines, columns=['project', 'algorithm', 'batch_size', 'builds_reqd', 'testall_size', 'delay_list', 'median_delay'])
    
    return df


In [26]:
filenames = ['combined_results.csv']
file_root = './'
dfs = []

for file in filenames:
    dfs.append(start_result_collection(file_root+file))

#combining dfs
final_df = pd.DataFrame()
final_df = dfs[0]

for i in range(1, len(dfs)):
    final_df = final_df.append(dfs[i])

final_df.to_csv('aici_results.csv')

1 5
1 5
3 7
8.0 15
1 5
1 5
3 7
8.0 15
1.0 4
1.0 4
3 7
7 15
1.0 4
1.0 4
3 7
7 15
1.0 3
1.0 3
3.5 7
7.5 15
1.0 2
1.0 3
3.5 7
7.5 15
2.0 6
2.0 6
3 7
7 15
1.0 6
2.0 6
3 7
7 15
1.0 3
1.0 3
4 7
7 15
1.0 2
1.0 3
4 7
7 15
2 3
2 3
4 7
8 15
1 1
2 3
4 7
8 15
1 6
1 6
4.0 7
8 15
1 5
1 6
4.0 7
8 15
1.0 7
1.0 7
3.0 7
7.0 15
1 7
1.0 7
3.0 7
7.0 15
2 5
2 5
3.0 7
8.0 15
1 4
2 5
3.0 7
8.0 15
1.0 5
1.0 5


/var/folders/09/2vzp5xpd2ldd4ggk4xzjt1qr0000gn/T/ipykernel_36844/513487934.py:9: DtypeWarning: Columns (4,7,10,14,19,41,59) have mixed types.Specify dtype option on import or set low_memory=False.
  aici_delay_lib = get_delay_lib(aici_results)


3 7
7.0 15
1 6
1.0 5
3 7
7.0 15
1 5
1 5
3.0 7
8 15
1 5
1 5
3.0 7
8 15
2.0 29
2.0 29
3.0 7
8.0 15
1.0 10
2.0 29
3.0 7
8.0 15
1.0 3
1.0 3
3 7
8 15
1 2
1.0 3
3 7
8 15
2 3
2 3
4 7
8.0 15
0.5 1
2 3
4 7
8.0 15
1 3
1 3
3.0 7
8.0 15
1 3
1 3
3.0 7
8.0 15
1 3
1 3
3 7
8.0 15
1.0 2
1 3
3 7
8.0 15
2.0 10
2.0 10
3.0 7
8 15
1.0 10
2.0 10
3.0 7
8 15
1.5 3
1.5 3
4.0 7
8 15
1 2
1.5 3
4.0 7
8 15
2.0 4
2.0 4
3 7
8 15
1 9
2.0 4
3 7
8 15
2 3
2 3
3.0 7
8 15
1.0 6
2 3
3.0 7
8 15


# Combining Versions

In [13]:
def separate_versions(results):
    version_dfs = []
    
    for i in range(1,11):
        ver = results[ results['version']==i]
        version_dfs.append(ver)
    return version_dfs

In [14]:
def combine_versions(results):
    
    projects = set(results['project'].tolist())
    all_versions = pd.DataFrame()
        
    for p in projects:
        
        p_data = results[ results['project']==p]
        pframe = pd.DataFrame()
        
        versions = separate_versions(p_data)
        
        for start in range(0,10):
            if len(versions[start]) > 0:
                pframe = versions[start]
                break
        
        for x in range(len(pframe)):
            row = pframe.iloc[x]
            
            alg = row['algorithm']
            b = row['batch_size']
            conf = row['confidence']
            
            final_delay_list = row['delay_list']
            
            for i in range(start+1,10):
                next_ver = versions[i]
                new_df = next_ver[ (next_ver['algorithm']==alg) & (next_ver['batch_size']==b) & (next_ver['confidence']==conf)]
                
                if len(new_df) > 0:
                    new_row = new_df.iloc[0]
                else:
                    continue
                
                total_size = row['testall_size'] + new_row['testall_size']
                total_reqd_builds = (row['builds_reqd']*row['testall_size']) + (new_row['builds_reqd']*new_row['testall_size'])
                final_delay_list.extend(new_row['delay_list'])
                
                row['builds_reqd'] = total_reqd_builds/total_size                
                row['delay_list'] = final_delay_list
                row['testall_size'] = total_size
                
            pframe.iloc[x] = row
        all_versions = all_versions.append(pframe)
    
    return all_versions

In [15]:
delay_df = combine_versions(final_df)


/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


In [16]:
delay_df.drop('delay_list', inplace=True, axis=1)

In [17]:
delay_df.to_csv('full_combined_delay.csv')